In [1]:
!pip install -U transformers datasets


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%cd ../

c:\Users\kstat\RuBLiMP


## An example for scoring an encoder LM on RuBLiMP

Here, we show how to score an encoder and decoder LM not decontaminated with RuBLiMP (that is, an external LM of interest) using a dataset sample.


In [3]:
from datasets import load_dataset


add_new_suffix = load_dataset("RussianNLP/rublimp", "add_new_suffix")
add_new_suffix = add_new_suffix["train"].to_pandas()
add_new_suffix.head(3)

c:\Users\kstat\RuBLiMP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,source_sentence,target_sentence,source_word,target_word,level,phenomenon,PID,subtype,domain,tree_depth
0,132968,Руками обхватила широкие плечи Окота.,Руками обхватила широкие плечники Окота.,плечи,плечники,Morphology,Word Formation,add_new_suffix,add_new_suffix,librusec,2
1,211801,Под левый сосок в междуреберье.,Под левый сосокщик в междуреберье.,сосок,сосокщик,Morphology,Word Formation,add_new_suffix,add_new_suffix,librusec,2
2,168527,Спуск темною ночью в Никиту...,Спуск темоватною ночью в Никиту...,темною,темоватною,Morphology,Word Formation,add_new_suffix,add_new_suffix,librusec,3


In [4]:
from src.scorer import *


encoder_model_name = "DeepPavlov/rubert-base-cased"
encoder_scorer = Scorer(model_name=encoder_model_name)

In [20]:
# scoring a dataset sample for illustration
sample = encoder_scorer.run(pool=add_new_suffix.sample(25))

Scoring with: DeepPavlov/rubert-base-cased: 100%|██████████| 25/25 [01:16<00:00,  3.04s/it]


In [21]:
# accuracy on the dataset sample
(
    sample["DeepPavlov-rubert-base-cased-ppl-s"]
    > sample["DeepPavlov-rubert-base-cased-ppl-t"]
).mean()

np.float64(0.96)

In [6]:
# for the whole dataset
sample = encoder_scorer.run(pool=add_new_suffix)

acc = (
    sample["DeepPavlov-rubert-base-cased-ppl-s"]
    > sample["DeepPavlov-rubert-base-cased-ppl-t"]
).mean()

print(f"Accuracy on 'add_new_suffix' with ruBERT-base: {acc:.3f}")

Scoring with: DeepPavlov/rubert-base-cased: 100%|██████████| 1000/1000 [1:16:20<00:00,  4.58s/it]  


Accuracy on 'add_new_suffix' with ruBERT-base: 0.925


In [8]:
!pip install huggingface_hub[hf_xet]


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from src.scorer import *
decoder_model_name = "ai-forever/rugpt3small_based_on_gpt2"
decoder_scorer = Scorer(model_name=decoder_model_name)

In [23]:
# scoring a dataset sample for illustration
sample = decoder_scorer.run(pool=add_new_suffix.sample(25))

Scoring with: ai-forever/rugpt3small_based_on_gpt2: 100%|██████████| 25/25 [00:03<00:00,  7.19it/s]


In [25]:
# accuracy on the dataset sample
(
    sample["rugpt-s-ppl-s"] < sample["rugpt-s-ppl-t"]
).mean()

np.float64(0.92)